# Prvi projekat

## Logistika
**Projekti se predaju preko github classroom sistema: https://classroom.github.com/a/E77gGtJ6**.

Napomene:
- Projekat se može raditi individualno ili u paru.
- U projektu je potrebno da se nalaze barem:
  - .py fajl sa skriptom koja sadrži tražene funkcije i čijim pokretanjem se izvršava celokupna analiza
    - Skripta ne mora da prima nikakve argumente (mada bi bilo lepše da se putanje do fajlova prosleđuju prilikom pokretanja, one mogu biti i eksplicitno zapisane ("hard-kodovane") u kodu).

  - README.md fajl u kojm navesti:
    - Imena studenata
    - Da li je su urađene sve tačke, i ako nisu šta nije urađeno
    - Da li je i na koji način korišćena pomoć AI alata  
    (što je dozvoljeno, ali se od studenata očekuje da razumeju i mogu da objasne sve što su dobili kao odgovor)

- Podatke (BCF, i TSV fajlove) nemojte dodavati u git repozitorijum! Obzirom na veličinu BCF fajla čak i slučano dodat, pa obrisan fajl će značajno povećati veličinu repozitorijuma i usporiti rad sa njim. Najbolje je na samom početku izrade napraviti .gitignore sa fajlovima u kojima su podaci, ili te fajlove držati izvan direktorijuma gde je kod.

- Dodatni fajlovi (Dockerfile, conda requiremnts i slično) se mogu, a ne moraju dodati u repozitorijum.

- Zadatak 4 nije neophodno uraditi kako bi se osvojilo svih 20 redovnih bodova. Studenti koji urade taj zadatak mogu na prvom projektu osvojiti više od 20 bodova, ali maksimalan zbir bodova na oba projekta ostaje ograničen na 40 (bonus bodovi služe da nadomeste eventualno izgubljene bodove).


## Postavka
Dati su fajlovi:
- BCF fajl sa čestim mutacijama iz projekta 1000 genoma (`1KG.subset.bcf`)
- Fajl sa oznakama fenotipa - zdravi i oboleli (`disease.tsv`)
- Fajl sa drugim fenotipskim podacima, gojaznost, pušenje i redovna fizička aktivnost (`env_factors.tsv`)
- Tabela ljudskih gena sa koordinatama u referentnom genomu (`entrez.hg38.tsv`)
- Tabela biolološkoh procesa sa povezanim genima (`kegg_pathways.tsv`)

##Zadaci:

1 [4 boda]. Napisati funkciju koja učitava VCF/BCF fajl u pandas Dataframe strukturu, pri čemu su kolone uzorci iz VCF/BCF fajla (jedan uzorak odgovora jednoj osobi), a kolone pozicije u genomu na kojim se nalaze mutacije. Imena kolona treba da budu imena uzoraka preuzeta iz VCF/BCF fajla. Indeks kolonu napraviti od hromozoma i pozicije (na primer u obliku "\<hromozom>-\<pozicija>"). Ćelije tabele treba da sadrže broj alternativnih alela za dati uzorak na datoj poziciji (0, 1 ili 2) i prema tipu treba da budu uint8. Pozicije koje imaju više od dva alela se preskaču prilikom pravljenja strukture.

.

2 [4 boda]. Napisati funkciju koja polazeći od pandas Dataframe-a iz zadataka 1 i pandas series objekat koji sadrži status osobe (1 - osoba je zahvaćena, na primer ima bolest, 0 - osoba nije zahvaćena, na primer nema bolest) izvršava GWAS analizu.
  - Primeniti hi-kvadrat test sa Laplace-ovom korekcijom (dodavanje 1 na svako polje, kako bi se izbegli problemi sa poljima gde je izbrojano 0 slučajeva).
  - Odabrati jedan GWAS model (dominantni, recesivni ili aditivni) i to dokumentovati u okviru funkcije (na primer kroz docstring)
  - Poželjno je da funckija interno koristi paralelizaciju (na primer kroz multiprocessing map). Vodite računa o efikasnosti izračunavanja. Za date podatke veoma efikasna implementacija se može završiti u roku od nekoliko minuta, za sproje može potrajati i nekoliko sati. Efikasnost ne utiče na ocenu, ali će dalja izrada i testiranje biti značajno lakši.
  - Rezulat funkcije treba da bude pandas Dataframe struktura sa jednom kolonom koja sadrži p vrednosti i indeksom koji je isti kao indeks ulazne Dataframe strukture sa genetskim podacima, sortirana po p vrednosti.
  - Funkcija treba da ima logički flag (True/False, podrazumevano True) na osnovu kog se rezultat filtrira tako da sadrži redove gde je p vrednost manja od zadate granične vrednosti, uz primenu Bonferoni korekcije.

.

3 [4 boda]. Napisati funkciju koja vrši grupisanje mutacija za koje postoji statistički značajna asocijacija, u strukturi iz tačke 3.
- Mogući algoritam grupisanja:
  - Odrediti kvadrat koeficijent korelacije genotipa za svaki par mutacija. Ukoliko lista `significant` sadrži listu gena mutacija koje hoćemo grupisati, a `genotypes` predstvalja početnu Dataframe strukturu mutacija x uzorak, kvadrat koeficijent korelacije za sve parove možemo odrediti kao:
  ```python
  np.corrcoef(genotypes[significant].T)**2
  ```
  - Za svaku mutaciju prolazimo kroz sve do tog trenutka formirane grupe i proveravamo da li je kvadrat koeficijent korelacije između bilo koje mutacije do sada dodate u datu grupu i mutacije koju obrađujemo preko zadate granične vrednosti (za potrebe projekta može se uzeti 0.2) i da li su te dve mutacije fizički dovoljno blizu (na istom hromozomu, i na primer unutar 600,000 baznih parova).
  - Ako je uslov zadovoljen za bilo koju mutaciju iz grupe, mutacija koja se obrađuje se dodaju u tu grupu (svaka mutacija može biti u samo jednoj grupi).
  - Ako ne postoji grupa kojoj mutacija pripada, formira se nova grupa sa tom mutacijom.
  - Rezultat funkcije treba da bude Dataframe struktura sa po jednim redom za svaku grupu i sledećim kolonama:
    - Vodeća mutacija (u formatu "\<hromozom>-\<pozicija>", bira se kao mutacija iz grupe čija je p vrednost najmanja)
    - P vrednost vodeće muatcija
    - Broj mutacija u hromozomu
    - Hromozom (sve mutacije u grupi su uvek na istom hromozomu)
    - Najmanja pozcija u grupi
    - Najveće pozicija u grupi
    - Raspon grupe duž horomozoma (najveća - najmanja pozicija)
    - Lista svih mutacija u grupi, u formatu "\<hromozom>-\<pozicija>"

.

4 [5 bodova - bonus]. Napraviti i evaluirati Bajesovu mrežu za predviđanje status osobe (zahvaćena / nije zahvaćena) na osnovu:
  - Genotipa vodećih mutacija iz prethodnog zadatka. Uzeti najviše 4 grupe sa najmanjim vrednostima vodeće mutacija.
  - Drugih fenotipskih podataka, iz dodatnog fajla
  - Genotipa vodećih mutacija i drugih fenotipskih podataka
- Za svaku od ovih mreža odrediti preciznost, odziv (recall) i tačnost (accuracy)

.

5 [4 boda]. Identifikovati gene koji se nalaze u regionima gde se su pronađene mutcije statistički značajno povezane sa statusom "bolesti".
  - Napisati funkciju koja kreira intervalska stabla na osnovu koordinata gena iz entrez tabele. Funkcija treba da vrati mapu čiji su ključevi identifikatori hromozoma (1, 2, 3... ili chr1, chr2, chr3...), a vrednosti stabla sa intervalima gena da datom hromozomu. Možete upotrebiti postojeću biblioteku za intervalska stabla (na primer intervaltree).
  - Napisati funkciju koja za svaku grupu iz zadatka 3 određuje gen u kojem se nalazi vodeća mutacija iz grupe.  

.

6 [4 boda]. Polazeći od gena identifikovanih u zadatku 5 odrediti biološke procese koji su povezani sa posmatranim stanjem. Jednostavan način kojim se ovo može odrediti je na osnovu procenta identifikovanih gena koji su označeni kao da učestvuju nekom procesu: $$score = \frac{PATHWAY\_GENES \cap GROUP\_GENES}{|GROUP\_GENES|} $$
Verovatno povezane biološke procese možemo odrediti tako što na ovako izračunat rezultat primenimo graničnu vrednost, na primer 0.7.

.

## Napomene
- Oznake hromozoma u VCF fajlu i entrez tabeli nisu direktno kompatibilni. Hromozomi u VCF fajlu su označeni kao chr1, chr2, ..., dok su u entrez tabeli samo brojevi 1, 2, ... Negde u kodu će biti potrebno da ove oznake "pomirite" (uklonite ili dodate prefiks chr).
- Liste gena povezanih sa procesima u tabeli `kegg_pathways.tsv` su dati kao jedan string, pri čemu su elemnti liste (pojedničani geni) odvojeni zapetom. U zadatku 56 je verovatno najbolje te stringove pretvoriti u liste (`string.split(',')`).
- Pravljenje Dataframe strukture u zadatku 1 će verovatno potrajati. Jednom napravljenu strukturu je korisno sačuvati (na primer u vidu feather fajla), kako bi kasnije izbegli ponovno pravljenje.
- Ukoliko projekat radite na OS X računaru, a želite da koriste multiprocessing za ubrzavanje izračunavanja, potrebno da izvršite sledći kod:
```python
import multiprocessing as mp
if mp.get_start_method() != 'fork':
    mp.set_start_method('fork', force=True)
```
- Možda je lakše projekat razvijati u notebook okruženju (jupyter ili unutar VS Code-a), pa portovati u .py skriptu (kako bi se izbeglo ponovno učitavanje / izračunavanje u toku rada)
- Biblioteka `tqdm` je veoma zgodna za praćenje rada petlji koje se dugo izvršavaju:
```python
from tqdm.auto import tqdm
```

## Task 1: Load VCF/BCF file into pandas DataFrame

This function loads a VCF/BCF file and converts it to a pandas DataFrame where:
- Columns are sample names from the VCF/BCF file
- Index is created from chromosome and position (format: "chr-position")  
- Cell values contain the number of alternative alleles (0, 1, or 2) as uint8
- Positions with more than 2 alleles are skipped

Expected output: DataFrame with genotype data ready for GWAS analysis

In [ ]:
!pip install pysam
!pip install intervaltree

import pandas as pd
import numpy as np
import pysam
from tqdm.auto import tqdm
import multiprocessing as mp
from scipy.stats import chi2_contingency
from intervaltree import IntervalTree
import os, subprocess, shutil
import warnings
warnings.filterwarnings('ignore')

def load_bcf_to_dataframe(bcf_file_path):
    """
    Load VCF/BCF file into pandas DataFrame.

    Parameters:
    bcf_file_path (str): Path to BCF file

    Returns:
    pd.DataFrame: DataFrame with samples as columns, positions as index,
                  containing alternative allele counts (0, 1, 2) as uint8
    """
    bcf_file = pysam.VariantFile(bcf_file_path)
    samples = list(bcf_file.header.samples)

    genotype_data = []
    positions = []

    for record in tqdm(bcf_file.fetch(), desc="Processing variants"):
        # Skip positions with more than 2 alleles
        if len(record.alleles) > 2:
            continue

        chrom = record.contig
        pos = record.pos
        position_id = f"{chrom}-{pos}"
        positions.append(position_id)

        # Extract genotype data for all samples
        genotypes = []
        for sample in samples:
            gt = record.samples[sample]['GT']
            if gt[0] is None or gt[1] is None:  # Missing data
                alt_count = 0
            else:
                alt_count = sum([1 for allele in gt if allele > 0])
            genotypes.append(alt_count)

        genotype_data.append(genotypes)

    # Create DataFrame
    df = pd.DataFrame(genotype_data, index=positions, columns=samples, dtype='uint8')

    bcf_file.close()
    return df

## Task 2: GWAS Analysis Function

This function performs Genome-Wide Association Study (GWAS) analysis using:
- Chi-square test with Laplace correction (adding 1 to each cell)
- Additive genetic model (0, 1, 2 alternative alleles)
- Multiprocessing for parallel computation
- Bonferroni correction for multiple testing
- Optional filtering by significance threshold

Expected output: DataFrame with p-values sorted by significance, optionally filtered

In [ ]:
def chi_square_test_single_variant(args):
    """Helper function for parallel processing of single variant"""
    position, genotypes, phenotypes = args

    # Create contingency table for additive model
    # Rows: phenotype (0=control, 1=case)
    # Columns: genotype (0, 1, 2 alternative alleles)
    contingency = np.zeros((2, 3), dtype=int)

    for i, (genotype, phenotype) in enumerate(zip(genotypes, phenotypes)):
        contingency[phenotype, genotype] += 1

    # Apply Laplace correction (add 1 to each cell)
    contingency += 1

    # Perform chi-square test
    try:
        chi2, p_value, dof, expected = chi2_contingency(contingency)
        return position, p_value
    except:
        return position, 1.0  # Return non-significant p-value if test fails

def gwas_analysis(genotype_df, phenotype_series, apply_bonferroni=True, significance_threshold=0.05, n_processes=None):
    """
    Perform GWAS analysis using chi-square test with Laplace correction.

    Uses additive genetic model where genotype values (0, 1, 2) represent
    the number of alternative alleles.

    Parameters:
    genotype_df (pd.DataFrame): Genotype data with positions as index, samples as columns
    phenotype_series (pd.Series): Phenotype status (0=control, 1=case) with samples as index
    apply_bonferroni (bool): Whether to apply Bonferroni correction and filter results
    significance_threshold (float): P-value threshold for significance
    n_processes (int): Number of processes for parallel computation

    Returns:
    pd.DataFrame: Results with p-values, sorted by significance
    """
    if n_processes is None:
        n_processes = mp.cpu_count()

    # Align genotype and phenotype data
    common_samples = genotype_df.columns.intersection(phenotype_series.index)
    genotype_aligned = genotype_df[common_samples]
    phenotype_aligned = phenotype_series[common_samples]

    # Prepare data for parallel processing
    args_list = []
    for position in genotype_aligned.index:
        genotypes = genotype_aligned.loc[position].values
        phenotypes = phenotype_aligned.values
        args_list.append((position, genotypes, phenotypes))

    # Parallel processing
    with mp.Pool(n_processes) as pool:
        results = list(tqdm(
            pool.imap(chi_square_test_single_variant, args_list),
            total=len(args_list),
            desc="GWAS analysis"
        ))

    # Create results DataFrame
    positions, p_values = zip(*results)
    results_df = pd.DataFrame({
        'p_value': p_values
    }, index=positions)

    # Sort by p-value
    results_df = results_df.sort_values('p_value')

    # Apply Bonferroni correction and filter if requested
    if apply_bonferroni:
        bonferroni_threshold = significance_threshold / len(results_df)
        results_df = results_df[results_df['p_value'] < bonferroni_threshold]

    return results_df

## Task 3: Group Statistically Significant Mutations

This function groups mutations based on:
- Squared correlation coefficient (r²) between genotypes (threshold: 0.2)
- Physical proximity on the same chromosome (within 600,000 base pairs)
- Each mutation belongs to only one group
- Groups are formed iteratively, checking all existing groups for each new mutation

Expected output: DataFrame with one row per group containing lead mutation, p-value, group statistics

In [ ]:
def group_significant_mutations(gwas_results, genotype_df, correlation_threshold=0.2, distance_threshold=600000):
    """
    Group statistically significant mutations based on correlation and physical distance.

    Parameters:
    gwas_results (pd.DataFrame): GWAS results with p-values
    genotype_df (pd.DataFrame): Genotype data with positions as index
    correlation_threshold (float): Minimum r² threshold for grouping
    distance_threshold (int): Maximum distance in base pairs for grouping

    Returns:
    pd.DataFrame: Groups with lead mutation, statistics, and member lists
    """
    if len(gwas_results) == 0:
        return pd.DataFrame()

    significant_positions = gwas_results.index.tolist()

    # Parse chromosome and position information
    def parse_position(pos_str):
        parts = pos_str.split('-')
        chrom = parts[0]
        pos = int(parts[1])
        return chrom, pos

    # Calculate correlation matrix for significant positions
    significant_genotypes = genotype_df.loc[significant_positions]
    correlation_matrix = np.corrcoef(significant_genotypes.values) ** 2

    # Initialize groups
    groups = []
    assigned = set()

    for i, current_pos in enumerate(tqdm(significant_positions, desc="Grouping mutations")):
        if current_pos in assigned:
            continue

        current_chrom, current_coord = parse_position(current_pos)
        current_group = [current_pos]
        assigned.add(current_pos)

        # Check if current position can be added to any existing group
        group_found = False
        for group in groups:
            can_join_group = False

            for member_pos in group:
                member_idx = significant_positions.index(member_pos)
                member_chrom, member_coord = parse_position(member_pos)

                # Check correlation and distance criteria
                if (correlation_matrix[i, member_idx] >= correlation_threshold and
                    current_chrom == member_chrom and
                    abs(current_coord - member_coord) <= distance_threshold):
                    can_join_group = True
                    break

            if can_join_group:
                group.extend(current_group)
                group_found = True
                break

        if not group_found:
            groups.append(current_group)

    # Create results DataFrame
    group_results = []
    for group in groups:
        # Find lead mutation (lowest p-value)
        group_p_values = gwas_results.loc[group, 'p_value']
        lead_mutation = group_p_values.idxmin()
        lead_p_value = group_p_values.min()

        # Parse positions and calculate statistics
        positions_data = [parse_position(pos) for pos in group]
        chromosomes = [chrom for chrom, pos in positions_data]
        coords = [pos for chrom, pos in positions_data]

        group_results.append({
            'lead_mutation': lead_mutation,
            'lead_p_value': lead_p_value,
            'num_mutations': len(group),
            'chromosome': chromosomes[0],  # All should be same chromosome
            'min_position': min(coords),
            'max_position': max(coords),
            'span': max(coords) - min(coords),
            'mutations': group
        })

    results_df = pd.DataFrame(group_results).sort_values('lead_p_value')
    return results_df

## Task 4 (Bonus): Bayesian Network for Disease Status Prediction

This creates and evaluates Bayesian networks for predicting disease status using:
1. Genotypes of lead mutations (top 4 groups with lowest p-values)
2. Environmental/phenotypic factors from additional file
3. Combined genotype and environmental data

Evaluation metrics: precision, recall, and accuracy for each network type

Expected output: Three trained Bayesian networks with performance metrics

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.preprocessing import LabelEncoder

def create_bayesian_networks(mutation_groups, genotype_df, phenotype_series, env_factors_file):
    """
    Create and evaluate Bayesian networks for disease status prediction.

    Parameters:
    mutation_groups (pd.DataFrame): Grouped mutations from task 3
    genotype_df (pd.DataFrame): Genotype data
    phenotype_series (pd.Series): Disease status
    env_factors_file (str): Path to environmental factors TSV file

    Returns:
    dict: Results with models and performance metrics
    """
    # Load environmental factors
    env_factors = pd.read_csv(env_factors_file, sep='\t', index_col=0)

    # Get top 4 lead mutations
    if len(mutation_groups) >= 4:
        top_mutations = mutation_groups.head(4)['lead_mutation'].tolist()
    else:
        top_mutations = mutation_groups['lead_mutation'].tolist()

    # Prepare datasets
    results = {}

    # 1. Genotype-only network
    if top_mutations:
        genotype_features = genotype_df.loc[top_mutations].T
        genotype_features.columns = [f'mutation_{i+1}' for i in range(len(top_mutations))]

        # Align with phenotype data
        common_samples = genotype_features.index.intersection(phenotype_series.index)
        X_genotype = genotype_features.loc[common_samples]
        y_genotype = phenotype_series.loc[common_samples]

        if len(X_genotype) > 0:
            results['genotype_only'] = train_and_evaluate_model(X_genotype, y_genotype, 'Genotype-only')

    # 2. Environmental factors only network
    common_samples_env = env_factors.index.intersection(phenotype_series.index)
    if len(common_samples_env) > 0:
        X_env = env_factors.loc[common_samples_env]
        y_env = phenotype_series.loc[common_samples_env]

        # Encode categorical variables if any
        X_env_encoded = X_env.copy()
        label_encoders = {}
        for column in X_env_encoded.columns:
            if X_env_encoded[column].dtype == 'object':
                le = LabelEncoder()
                X_env_encoded[column] = le.fit_transform(X_env_encoded[column].astype(str))
                label_encoders[column] = le

        results['environmental_only'] = train_and_evaluate_model(X_env_encoded, y_env, 'Environmental-only')

    # 3. Combined network
    if top_mutations and len(common_samples_env) > 0:
        # Find common samples across all datasets
        common_all = (genotype_features.index
                     .intersection(env_factors.index)
                     .intersection(phenotype_series.index))

        if len(common_all) > 0:
            X_combined = pd.concat([
                genotype_features.loc[common_all],
                X_env_encoded.loc[common_all]
            ], axis=1)
            y_combined = phenotype_series.loc[common_all]

            results['combined'] = train_and_evaluate_model(X_combined, y_combined, 'Combined')

    return results

def train_and_evaluate_model(X, y, model_name):
    """
    Train and evaluate a Gaussian Naive Bayes model.

    Parameters:
    X (pd.DataFrame): Features
    y (pd.Series): Target variable
    model_name (str): Name for reporting

    Returns:
    dict: Model and performance metrics
    """
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )

    # Train Gaussian Naive Bayes (approximation of Bayesian Network)
    model = GaussianNB()
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate metrics
    precision = precision_score(y_test, y_pred, average='binary')
    recall = recall_score(y_test, y_pred, average='binary')
    accuracy = accuracy_score(y_test, y_pred)

    return {
        'model': model,
        'precision': precision,
        'recall': recall,
        'accuracy': accuracy,
        'model_name': model_name,
        'n_features': X.shape[1],
        'n_samples': len(X)
    }

## Task 5: Identify Genes in Significant Regions

This creates interval trees for gene coordinates and identifies genes containing lead mutations:
1. Build interval trees per chromosome from gene coordinate data
2. Handle chromosome naming differences (chr1 vs 1)
3. Query trees to find genes overlapping with lead mutation positions

Expected output: Functions for creating interval trees and finding genes containing mutations

In [ ]:
def create_gene_interval_trees(entrez_file):
    """
    Create interval trees for gene coordinates from entrez gene table.

    Parameters:
    entrez_file (str): Path to entrez gene TSV file

    Returns:
    dict: Chromosome ID -> IntervalTree mapping
    """
    genes_df = pd.read_csv(entrez_file, sep='\t')

    # Create interval trees for each chromosome
    chromosome_trees = {}

    for _, gene in genes_df.iterrows():
        chrom = str(gene['chromosome'])
        start = gene['start']
        end = gene['end']
        gene_id = gene['gene_id']
        gene_symbol = gene.get('gene_symbol', f'Gene_{gene_id}')

        # Normalize chromosome naming (remove 'chr' prefix if present)
        if chrom.startswith('chr'):
            chrom = chrom[3:]

        # Create tree for chromosome if it doesn't exist
        if chrom not in chromosome_trees:
            chromosome_trees[chrom] = IntervalTree()

        # Add gene interval to tree (store gene info as data)
        chromosome_trees[chrom][start:end+1] = {
            'gene_id': gene_id,
            'gene_symbol': gene_symbol,
            'start': start,
            'end': end
        }

    return chromosome_trees

def find_genes_for_mutations(mutation_groups, chromosome_trees):
    """
    Find genes containing lead mutations from each group.

    Parameters:
    mutation_groups (pd.DataFrame): Grouped mutations from task 3
    chromosome_trees (dict): Chromosome interval trees from create_gene_interval_trees

    Returns:
    pd.DataFrame: Mutation groups with associated gene information
    """
    def parse_position(pos_str):
        parts = pos_str.split('-')
        chrom = parts[0]
        pos = int(parts[1])
        return chrom, pos

    def normalize_chromosome(chrom):
        # Remove 'chr' prefix to match entrez table format
        if chrom.startswith('chr'):
            return chrom[3:]
        return chrom

    results = []

    for _, group in mutation_groups.iterrows():
        lead_mutation = group['lead_mutation']
        chrom, pos = parse_position(lead_mutation)
        norm_chrom = normalize_chromosome(chrom)

        # Find overlapping genes
        overlapping_genes = []
        if norm_chrom in chromosome_trees:
            overlaps = chromosome_trees[norm_chrom][pos]
            for interval in overlaps:
                gene_info = interval.data
                overlapping_genes.append(gene_info)

        # Create result entry
        result = group.to_dict()
        if overlapping_genes:
        # If multiple genes overlap, take the first one
            # (in practice, this is rare for point mutations)
            gene_info = overlapping_genes[0]
            result['gene_id'] = gene_info['gene_id']
            result['gene_symbol'] = gene_info['gene_symbol']
            result['gene_start'] = gene_info['start']
            result['gene_end'] = gene_info['end']
        else:
            result['gene_id'] = None
            result['gene_symbol'] = None
            result['gene_start'] = None
            result['gene_end'] = None

        result['num_overlapping_genes'] = len(overlapping_genes)
        results.append(result)
        return pd.DataFrame(results)

## Task 6: Biological Process Analysis

This identifies biological processes connected to genes from significant mutation regions:
1. Load KEGG pathway data and parse gene lists (comma-separated strings)
2. Calculate overlap score: (pathway_genes ∩ identified_genes) / |identified_genes|
3. Apply threshold (e.g., 0.7) to identify likely connected biological processes

Expected output: Function to analyze pathway enrichment and identify relevant biological processes

In [ ]:
def analyze_biological_processes(genes_with_mutations, kegg_pathways_file, score_threshold=0.7):
    """
    Identify biological processes connected to identified genes.

    Parameters:
    genes_with_mutations (pd.DataFrame): Results from find_genes_for_mutations
    kegg_pathways_file (str): Path to KEGG pathways TSV file
    score_threshold (float): Minimum overlap score for pathway significance

    Returns:
    pd.DataFrame: Pathways with overlap scores above threshold
    """
    # Load KEGG pathways
    pathways_df = pd.read_csv(kegg_pathways_file, sep='\t')

    # Extract identified genes (exclude None values)
    identified_genes = set()
    for _, row in genes_with_mutations.iterrows():
        if pd.notna(row['gene_id']):
            identified_genes.add(str(row['gene_id']))
        if pd.notna(row['gene_symbol']):
            identified_genes.add(str(row['gene_symbol']))

    if not identified_genes:
        print("No genes identified from mutations")
        return pd.DataFrame()

    print(f"Analyzing {len(identified_genes)} identified genes: {identified_genes}")

    # Analyze each pathway
    pathway_results = []

    for _, pathway in pathways_df.iterrows():
        pathway_id = pathway.get('pathway_id', pathway.get('pathway', 'Unknown'))
        pathway_name = pathway.get('pathway_name', pathway.get('description', 'Unknown'))
        genes_str = pathway.get('genes', pathway.get('gene_list', ''))

        if pd.isna(genes_str) or genes_str == '':
            continue

        # Parse gene list (comma-separated)
        pathway_genes = set([gene.strip() for gene in str(genes_str).split(',') if gene.strip()])

        if not pathway_genes:
            continue

        # Calculate overlap
        overlap_genes = identified_genes.intersection(pathway_genes)
        overlap_count = len(overlap_genes)

        # Calculate score
        score = overlap_count / len(identified_genes) if identified_genes else 0

        pathway_results.append({
            'pathway_id': pathway_id,
            'pathway_name': pathway_name,
            'overlap_score': score,
            'overlap_count': overlap_count,
            'identified_genes_count': len(identified_genes),
            'pathway_genes_count': len(pathway_genes),
            'overlapping_genes': list(overlap_genes),
            'all_pathway_genes': list(pathway_genes)
        })

    # Create results DataFrame
    results_df = pd.DataFrame(pathway_results)

    if len(results_df) == 0:
        print("No pathway results generated")
        return pd.DataFrame()

    # Filter by threshold and sort by score
    significant_pathways = results_df[results_df['overlap_score'] >= score_threshold].copy()
    significant_pathways = significant_pathways.sort_values('overlap_score', ascending=False)

    print(f"Found {len(significant_pathways)} pathways above threshold {score_threshold}")

    return significant_pathways

def print_pathway_analysis_summary(pathway_results, genes_with_mutations):
    """
    Print a summary of the pathway analysis results.

    Parameters:
    pathway_results (pd.DataFrame): Results from analyze_biological_processes
    genes_with_mutations (pd.DataFrame): Gene information from mutations
    """
    print("\\n" + "="*60)
    print("BIOLOGICAL PROCESS ANALYSIS SUMMARY")
    print("="*60)

    # Summary of identified genes
    identified_genes = []
    for _, row in genes_with_mutations.iterrows():
        if pd.notna(row['gene_symbol']):
            identified_genes.append(row['gene_symbol'])
        elif pd.notna(row['gene_id']):
            identified_genes.append(str(row['gene_id']))

    print(f"\\nIdentified genes from significant mutations: {len(set(identified_genes))}")
    print(f"Genes: {', '.join(set(identified_genes))}")

    # Summary of pathway results
    if len(pathway_results) > 0:
        print(f"\\nSignificant biological processes: {len(pathway_results)}")
        print("\\nTop pathways:")
        for _, pathway in pathway_results.head(10).iterrows():
            print(f"  - {pathway['pathway_name']} (score: {pathway['overlap_score']:.3f})")
    else:
        print("\\nNo significant biological processes identified.")

    print("="*60)

## Main Analysis Pipeline

This section demonstrates how to run the complete analysis pipeline with all implemented functions.
The pipeline loads data, performs GWAS analysis, groups mutations, identifies genes, and analyzes biological processes.

Expected workflow: Complete bioinformatics analysis from BCF file to biological insights

In [ ]:
# Main analysis pipeline
def run_complete_analysis():
    """
    Run the complete bioinformatics analysis pipeline.
    """
    base_drive = None
    try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=False)
        base_drive = "/content/drive/MyDrive/bioinformatika"
    except Exception:
        pass


    def first_existing(*paths):
        for p in paths:
            if p and os.path.exists(p):
                return p
        return None


    bcf_file = first_existing("1KG.subset.bcf",
                              f"{base_drive}/1KG.subset.bcf" if base_drive else None)
    disease_file = first_existing("disease.tsv",
                                  f"{base_drive}/disease.tsv" if base_drive else None)
    env_factors_file = first_existing("env_factors.tsv",
                                      f"{base_drive}/env_factors.tsv" if base_drive else None)
    entrez_file = first_existing("entrez.hg38.tsv",
                                 f"{base_drive}/entrez.hg38.tsv" if base_drive else None)
    kegg_file = first_existing("kegg_pathways.tsv",
                               f"{base_drive}/kegg_pathways.tsv" if base_drive else None)


    missing = { "BCF": bcf_file, "disease": disease_file, "env_factors": env_factors_file,
                "entrez": entrez_file, "kegg": kegg_file }
    not_found = [k for k,v in missing.items() if v is None]
    if not_found:
        raise FileNotFoundError(
            "Nisam našao sledeće fajlove (ni lokalno ni na Drive-u /MyDrive/bioinformatika): "
            + ", ".join(not_found)
        )

    print("Starting complete bioinformatics analysis pipeline...")
    print("="*60)

    # Task 1: Load BCF file
    print("\\n1. Loading BCF file...")
    genotype_df = load_bcf_to_dataframe(bcf_file)
    print(f"Loaded genotype data: {genotype_df.shape[0]} variants, {genotype_df.shape[1]} samples")

    # Save genotype data for future use
    # genotype_df.to_feather("genotype_data.feather")

    # Load phenotype data
    print("\\n2. Loading phenotype data...")
    phenotype_df = pd.read_csv(disease_file, sep='\\t', index_col=0)
    phenotype_series = phenotype_df.iloc[:, 0]  # Assuming first column is disease status
    print(f"Loaded phenotype data: {len(phenotype_series)} samples")

    # Task 2: GWAS analysis
    print("\\n3. Performing GWAS analysis...")
    gwas_results = gwas_analysis(genotype_df, phenotype_series)
    print(f"GWAS completed: {len(gwas_results)} significant associations found")

    if len(gwas_results) > 0:
        # Task 3: Group mutations
        print("\\n4. Grouping significant mutations...")
        mutation_groups = group_significant_mutations(gwas_results, genotype_df)
        print(f"Formed {len(mutation_groups)} mutation groups")

        # Task 5: Identify genes
        print("\\n5. Identifying genes in significant regions...")
        chromosome_trees = create_gene_interval_trees(entrez_file)
        genes_with_mutations = find_genes_for_mutations(mutation_groups, chromosome_trees)
        print(f"Identified genes for {len(genes_with_mutations)} groups")

        # Task 6: Biological process analysis
        print("\\n6. Analyzing biological processes...")
        pathway_results = analyze_biological_processes(genes_with_mutations, kegg_file)
        print_pathway_analysis_summary(pathway_results, genes_with_mutations)

        # Task 4: Bayesian networks
        print("\\n7. Creating Bayesian networks (bonus task)...")
        try:
            bn_results = create_bayesian_networks(mutation_groups, genotype_df, phenotype_series, env_factors_file)
            print("\\nBayesian Network Results:")
            for model_type, results in bn_results.items():
                print(f"  {results['model_name']}:")
                print(f"    Precision: {results['precision']:.3f}")
                print(f"    Recall: {results['recall']:.3f}")
                print(f"    Accuracy: {results['accuracy']:.3f}")
        except Exception as e:
            print(f"Bayesian networks failed: {e}")

        print("\\n" + "="*60)
        print("Analysis completed successfully!")
        print("="*60)

        return {
            'genotype_df': genotype_df,
            'gwas_results': gwas_results,
            'mutation_groups': mutation_groups,
            'genes_with_mutations': genes_with_mutations,
            'pathway_results': pathway_results
        }
    else:
        print("No significant associations found. Analysis stopped.")
        return None


results = run_complete_analysis()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Starting complete bioinformatics analysis pipeline...
\n1. Loading BCF file...


Processing variants: 0it [00:00, ?it/s]